In [5]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import matplotlib
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
data_path = os.chdir("C:\\Users\\NoMad\\PycharmProjects\\SANEM\\movements")

In [21]:
# другой вариант таблицы, где 25 - число движений, 24 - число пространственных переменных, n - число моментов, потребовавшихся для записи движения
# (25, 24, n)
x_train = np.array(pd.read_csv('m01_s01_e01_positions.txt', sep=','))
x_train = np.array([x_train[:, 42:].transpose()])
print(x_train.shape)

for i in range(1,7):
    for j in range(2,6):
        data1 = np.array(pd.read_csv(f"m0{i}_s0{j}_e01_positions.txt", sep=','))
        data1 = np.array([data1[:, 42:].transpose()])

        if(x_train.shape[2]>data1.shape[2]):
            x_train = x_train[:,:,:data1.shape[2]]
            x_train = np.append(x_train, data1, axis=0)
        elif(x_train.shape[2]<data1.shape[2]):
            data1 = data1[:,:,:x_train.shape[2]]
            x_train = np.append(x_train, data1, axis=0)
        elif(x_train.shape[2]==data1.shape[2]):
            x_train = np.append(x_train, data1, axis=0)

print(x_train.shape)

(1, 24, 72)
(25, 24, 51)


In [22]:
x_train = x_train.reshape(x_train.shape[0], 24, x_train.shape[2], 1)
x_train = x_train.astype('float32')
print(x_train.shape)

(25, 24, 51, 1)


In [104]:
# проверочные данные:
predict_move_1 = np.array(pd.read_csv('m01_s01_e05_positions.txt', sep=','))
predict_move_1 = np.array([predict_move_1[:51, 42:].transpose()])
print(predict_move_1.shape)
predict_move_1 = predict_move_1.reshape(predict_move_1.shape[0], 24, predict_move_1.shape[2], 1)
predict_move_1 = predict_move_1.astype('float32')
print("predict_move_1 = ",predict_move_1.shape)

predict_move_2 = np.array(pd.read_csv('m02_s01_e01_positions.txt', sep=','))
predict_move_2 = np.array([predict_move_2[:51, 42:].transpose()])
print(predict_move_2.shape)
predict_move_2 = predict_move_2.reshape(predict_move_2.shape[0], 24, predict_move_2.shape[2], 1)
predict_move_2 = predict_move_2.astype('float32')
print("predict_move_2 = ",predict_move_2.shape)

predict_move_3 = np.array(pd.read_csv('m03_s02_e02_positions.txt', sep=','))
predict_move_3 = np.array([predict_move_3[:51, 42:].transpose()])
print(predict_move_3.shape)
predict_move_3 = predict_move_3.reshape(predict_move_3.shape[0], 24, predict_move_3.shape[2], 1)
predict_move_3 = predict_move_3.astype('float32')
print("predict_move_3 = ",predict_move_3.shape)

predict_move_4 = np.array(pd.read_csv('m04_s01_e01_positions.txt', sep=','))
predict_move_4 = np.array([predict_move_4[:51, 42:].transpose()])
print(predict_move_4.shape)
predict_move_4 = predict_move_4.reshape(predict_move_4.shape[0], 24, predict_move_4.shape[2], 1)
predict_move_4 = predict_move_4.astype('float32')
print("predict_move_4 = ",predict_move_4.shape)

predict_move_5 = np.array(pd.read_csv('m05_s01_e01_positions.txt', sep=','))
predict_move_5 = np.array([predict_move_5[:51, 42:].transpose()])
print(predict_move_5.shape)
predict_move_5 = predict_move_5.reshape(predict_move_5.shape[0], 24, predict_move_5.shape[2], 1)
predict_move_5 = predict_move_5.astype('float32')
print("predict_move_5 = ",predict_move_5.shape)

predict_move_6 = np.array(pd.read_csv('m06_s01_e05_positions.txt', sep=','))
predict_move_6 = np.array([predict_move_6[:51, 42:].transpose()])
print(predict_move_6.shape)
predict_move_6 = predict_move_6.reshape(predict_move_6.shape[0], 24, predict_move_6.shape[2], 1)
predict_move_6 = predict_move_6.astype('float32')
print("predict_move_6 = ",predict_move_6.shape)

(1, 24, 51)
predict_move_1 =  (1, 24, 51, 1)
(1, 24, 51)
predict_move_2 =  (1, 24, 51, 1)
(1, 24, 51)
predict_move_3 =  (1, 24, 51, 1)
(1, 24, 51)
predict_move_4 =  (1, 24, 51, 1)
(1, 24, 51)
predict_move_5 =  (1, 24, 51, 1)
(1, 24, 51)
predict_move_6 =  (1, 24, 51, 1)


In [11]:
# выходной набор данных. Столбец с единицей соответствует номеру соответствующего движения (движений всего 6) 
# y_train = [[1,0,0,0,0,0], [1,0,0,0,0,0], [1,0,0,0,0,0], [1,0,0,0,0,0], [1,0,0,0,0,0],
#            [0,1,0,0,0,0], [0,1,0,0,0,0], [0,1,0,0,0,0], [0,1,0,0,0,0],
#            [0,0,1,0,0,0], [0,0,1,0,0,0], [0,0,1,0,0,0], [0,0,1,0,0,0],
#            [0,0,0,1,0,0], [0,0,0,1,0,0], [0,0,0,1,0,0], [0,0,0,1,0,0],
#            [0,0,0,0,1,0], [0,0,0,0,1,0], [0,0,0,0,1,0], [0,0,0,0,1,0],
#            [0,0,0,0,0,1], [0,0,0,0,0,1], [0,0,0,0,0,1]]

y_train = [0,0,0,0,0,1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5]
y_train = np.array(y_train, dtype=np.float32)
y_train = keras.utils.to_categorical(y_train)
print(y_train[:6])

[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]


In [76]:
# другой вариант нейросети (сверточная)
input = keras.Input(shape=(24,x_train.shape[2],1))
inputs = layers.Convolution2D(25, input_shape=(24,), kernel_size=(3,1), activation="relu")(input)
hidden1 = layers.Convolution2D(25, kernel_size=(3,1), activation="softmax")(inputs)
hidden1 = layers.Dropout(0.2)(hidden1)
h1 = layers.Flatten()(hidden1)
outputs = layers.Dense(6, activation="sigmoid")(h1)

seq_model = keras.Model(inputs=input, outputs=outputs, name="sequential_or_what")
seq_model.summary()

Model: "sequential_or_what"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 24, 51, 1)]       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 22, 51, 25)        100       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 20, 51, 25)        1900      
_________________________________________________________________
dropout_4 (Dropout)          (None, 20, 51, 25)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 25500)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 153006    
Total params: 155,006
Trainable params: 155,006
Non-trainable params: 0
__________________________________________

In [77]:
seq_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

In [113]:
seq_model.fit(x_train,y_train,epochs=1000,batch_size=2)

Epoch 1/1000
13/13 [==============================] - 0s 8ms/step - loss: 4.5503e-04 - accuracy: 1.0000
Epoch 2/1000
13/13 [==============================] - 0s 7ms/step - loss: 2.6274e-04 - accuracy: 1.0000
Epoch 3/1000
13/13 [==============================] - 0s 8ms/step - loss: 3.6472e-04 - accuracy: 1.0000
Epoch 4/1000
13/13 [==============================] - 0s 8ms/step - loss: 3.2109e-04 - accuracy: 1.0000
Epoch 5/1000
13/13 [==============================] - 0s 8ms/step - loss: 5.3424e-04 - accuracy: 1.0000
Epoch 6/1000
13/13 [==============================] - 0s 8ms/step - loss: 3.0404e-04 - accuracy: 1.0000
Epoch 7/1000
13/13 [==============================] - 0s 8ms/step - loss: 4.2453e-04 - accuracy: 1.0000
Epoch 8/1000
13/13 [==============================] - 0s 8ms/step - loss: 3.4257e-04 - accuracy: 1.0000
Epoch 9/1000
13/13 [==============================] - 0s 8ms/step - loss: 3.6731e-04 - accuracy: 1.0000
Epoch 10/1000
13/13 [==============================] - 0s 8ms/st

13/13 [==============================] - 0s 9ms/step - loss: 2.2388e-04 - accuracy: 1.0000
Epoch 79/1000
13/13 [==============================] - 0s 9ms/step - loss: 2.1001e-04 - accuracy: 1.0000
Epoch 80/1000
13/13 [==============================] - 0s 8ms/step - loss: 2.0523e-04 - accuracy: 1.0000
Epoch 81/1000
13/13 [==============================] - 0s 7ms/step - loss: 3.0167e-04 - accuracy: 1.0000
Epoch 82/1000
13/13 [==============================] - 0s 8ms/step - loss: 1.9472e-04 - accuracy: 1.0000
Epoch 83/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.2403e-04 - accuracy: 1.0000
Epoch 84/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.4824e-04 - accuracy: 1.0000
Epoch 85/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.6339e-04 - accuracy: 1.0000
Epoch 86/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.9444e-04 - accuracy: 1.0000
Epoch 87/1000
13/13 [==============================] - 0s 7ms/step - 

13/13 [==============================] - 0s 8ms/step - loss: 2.7178e-04 - accuracy: 1.0000
Epoch 157/1000
13/13 [==============================] - 0s 8ms/step - loss: 1.6801e-04 - accuracy: 1.0000
Epoch 158/1000
13/13 [==============================] - 0s 7ms/step - loss: 4.1988e-04 - accuracy: 1.0000
Epoch 159/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.8652e-04 - accuracy: 1.0000
Epoch 160/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.4358e-04 - accuracy: 1.0000
Epoch 161/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.7522e-04 - accuracy: 1.0000
Epoch 162/1000
13/13 [==============================] - 0s 7ms/step - loss: 2.3752e-04 - accuracy: 1.0000
Epoch 163/1000
13/13 [==============================] - 0s 8ms/step - loss: 2.8633e-04 - accuracy: 1.0000
Epoch 164/1000
13/13 [==============================] - 0s 8ms/step - loss: 1.9057e-04 - accuracy: 1.0000
Epoch 165/1000
13/13 [==============================] - 0s 7m

13/13 [==============================] - 0s 7ms/step - loss: 1.1283e-04 - accuracy: 1.0000
Epoch 234/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.5596e-04 - accuracy: 1.0000
Epoch 235/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.0209e-04 - accuracy: 1.0000
Epoch 236/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.9688e-04 - accuracy: 1.0000
Epoch 237/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.1387e-04 - accuracy: 1.0000
Epoch 238/1000
13/13 [==============================] - 0s 7ms/step - loss: 8.6782e-05 - accuracy: 1.0000
Epoch 239/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.1306e-04 - accuracy: 1.0000
Epoch 240/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.1826e-04 - accuracy: 1.0000
Epoch 241/1000
13/13 [==============================] - 0s 7ms/step - loss: 2.3246e-04 - accuracy: 1.0000
Epoch 242/1000
13/13 [==============================] - 0s 7m

13/13 [==============================] - 0s 7ms/step - loss: 7.0711e-05 - accuracy: 1.0000
Epoch 311/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.2967e-04 - accuracy: 1.0000
Epoch 312/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.0429e-04 - accuracy: 1.0000
Epoch 313/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.2929e-04 - accuracy: 1.0000
Epoch 314/1000
13/13 [==============================] - 0s 7ms/step - loss: 6.6990e-05 - accuracy: 1.0000
Epoch 315/1000
13/13 [==============================] - 0s 7ms/step - loss: 6.7111e-05 - accuracy: 1.0000
Epoch 316/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.2467e-04 - accuracy: 1.0000
Epoch 317/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.2428e-04 - accuracy: 1.0000
Epoch 318/1000
13/13 [==============================] - 0s 7ms/step - loss: 7.8297e-05 - accuracy: 1.0000
Epoch 319/1000
13/13 [==============================] - 0s 7m

...

13/13 [==============================] - 0s 7ms/step - loss: 1.4651e-05 - accuracy: 1.0000
Epoch 619/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.5451e-05 - accuracy: 1.0000
Epoch 620/1000
13/13 [==============================] - 0s 7ms/step - loss: 2.3275e-05 - accuracy: 1.0000
Epoch 621/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.7520e-05 - accuracy: 1.0000
Epoch 622/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.4749e-05 - accuracy: 1.0000
Epoch 623/1000
13/13 [==============================] - 0s 7ms/step - loss: 9.3904e-06 - accuracy: 1.0000
Epoch 624/1000
13/13 [==============================] - 0s 7ms/step - loss: 2.2921e-05 - accuracy: 1.0000
Epoch 625/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.0579e-05 - accuracy: 1.0000
Epoch 626/1000
13/13 [==============================] - 0s 7ms/step - loss: 2.2113e-05 - accuracy: 1.0000
Epoch 627/1000
13/13 [==============================] - 0s 7m

13/13 [==============================] - 0s 7ms/step - loss: 2.4208e-05 - accuracy: 1.0000
Epoch 696/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.0222e-05 - accuracy: 1.0000
Epoch 697/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.3844e-05 - accuracy: 1.0000
Epoch 698/1000
13/13 [==============================] - 0s 7ms/step - loss: 2.0680e-05 - accuracy: 1.0000
Epoch 699/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.8534e-05 - accuracy: 1.0000
Epoch 700/1000
13/13 [==============================] - 0s 8ms/step - loss: 1.9042e-05 - accuracy: 1.0000
Epoch 701/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.0318e-05 - accuracy: 1.0000
Epoch 702/1000
13/13 [==============================] - 0s 7ms/step - loss: 8.9911e-06 - accuracy: 1.0000
Epoch 703/1000
13/13 [==============================] - 0s 7ms/step - loss: 9.8952e-06 - accuracy: 1.0000
Epoch 704/1000
13/13 [==============================] - 0s 8m

13/13 [==============================] - 0s 8ms/step - loss: 7.5697e-06 - accuracy: 1.0000
Epoch 773/1000
13/13 [==============================] - 0s 8ms/step - loss: 1.1038e-05 - accuracy: 1.0000
Epoch 774/1000
13/13 [==============================] - 0s 8ms/step - loss: 7.2109e-06 - accuracy: 1.0000
Epoch 775/1000
13/13 [==============================] - 0s 7ms/step - loss: 7.8557e-06 - accuracy: 1.0000
Epoch 776/1000
13/13 [==============================] - 0s 7ms/step - loss: 4.3711e-06 - accuracy: 1.0000
Epoch 777/1000
13/13 [==============================] - 0s 7ms/step - loss: 6.9786e-06 - accuracy: 1.0000
Epoch 778/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.0984e-05 - accuracy: 1.0000
Epoch 779/1000
13/13 [==============================] - 0s 8ms/step - loss: 1.4978e-05 - accuracy: 1.0000
Epoch 780/1000
13/13 [==============================] - 0s 8ms/step - loss: 6.4474e-06 - accuracy: 1.0000
Epoch 781/1000
13/13 [==============================] - 0s 7m

13/13 [==============================] - 0s 7ms/step - loss: 4.0592e-05 - accuracy: 1.0000
Epoch 850/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.0604e-04 - accuracy: 1.0000
Epoch 851/1000
13/13 [==============================] - 0s 7ms/step - loss: 5.9051e-05 - accuracy: 1.0000
Epoch 852/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.0738e-04 - accuracy: 1.0000
Epoch 853/1000
13/13 [==============================] - 0s 7ms/step - loss: 7.2071e-05 - accuracy: 1.0000
Epoch 854/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.3846e-04 - accuracy: 1.0000
Epoch 855/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.4574e-04 - accuracy: 1.0000
Epoch 856/1000
13/13 [==============================] - 0s 7ms/step - loss: 8.5651e-05 - accuracy: 1.0000
Epoch 857/1000
13/13 [==============================] - 0s 7ms/step - loss: 7.6045e-05 - accuracy: 1.0000
Epoch 858/1000
13/13 [==============================] - 0s 7m

13/13 [==============================] - 0s 7ms/step - loss: 3.9753e-05 - accuracy: 1.0000
Epoch 927/1000
13/13 [==============================] - 0s 7ms/step - loss: 4.3655e-05 - accuracy: 1.0000
Epoch 928/1000
13/13 [==============================] - 0s 7ms/step - loss: 6.3233e-05 - accuracy: 1.0000
Epoch 929/1000
13/13 [==============================] - 0s 7ms/step - loss: 7.8057e-05 - accuracy: 1.0000
Epoch 930/1000
13/13 [==============================] - 0s 8ms/step - loss: 4.6622e-05 - accuracy: 1.0000
Epoch 931/1000
13/13 [==============================] - 0s 7ms/step - loss: 2.1569e-05 - accuracy: 1.0000
Epoch 932/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.2844e-04 - accuracy: 1.0000
Epoch 933/1000
13/13 [==============================] - 0s 8ms/step - loss: 2.3131e-05 - accuracy: 1.0000
Epoch 934/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.8787e-04 - accuracy: 1.0000
Epoch 935/1000
13/13 [==============================] - 0s 8m

In [107]:
predictArr = seq_model.predict(predict_move_1)
print(predictArr[0])

maxPercentage = 0
for i in enumerate(predictArr[0]):
    if(i[1]>maxPercentage):
        maxPercentage=i[1]
        move=i[0]+1     
print(round(maxPercentage*100, 2),"% this is move № ", move)

[9.9956077e-01 2.2152633e-09 1.7845155e-03 3.7216330e-05 2.2841431e-04
 5.6621627e-07]
99.96 % this is move №  1


In [108]:
predictArr = seq_model.predict(predict_move_2)
print(predictArr[0])

maxPercentage = 0
for i in enumerate(predictArr[0]):
    if(i[1]>maxPercentage):
        maxPercentage=i[1]
        move=i[0]+1     
print(round(maxPercentage*100, 2),"% this is move № ", move)

[2.0844756e-04 9.8544794e-01 1.1786154e-05 1.6733770e-06 3.3644701e-05
 2.1053691e-04]
98.54 % this is move №  2


In [109]:
predictArr = seq_model.predict(predict_move_3)
print(predictArr[0])

maxPercentage = 0
for i in enumerate(predictArr[0]):
    if(i[1]>maxPercentage):
        maxPercentage=i[1]
        move=i[0]+1     
print(round(maxPercentage*100, 2),"% this is move № ", move)

[8.3974458e-02 7.8731747e-03 7.1645075e-01 1.1259152e-09 4.1048992e-05
 7.2413845e-06]
71.65 % this is move №  3


In [110]:
predictArr = seq_model.predict(predict_move_4)
print(predictArr[0])

maxPercentage = 0
for i in enumerate(predictArr[0]):
    if(i[1]>maxPercentage):
        maxPercentage=i[1]
        move=i[0]+1     
print(round(maxPercentage*100, 2),"% this is move № ", move)

[1.6539551e-04 6.1365348e-08 5.8358400e-03 9.9485070e-01 1.3938559e-03
 9.3933318e-07]
99.49 % this is move №  4


In [111]:
predictArr = seq_model.predict(predict_move_5)
print(predictArr[0])

maxPercentage = 0
for i in enumerate(predictArr[0]):
    if(i[1]>maxPercentage):
        maxPercentage=i[1]
        move=i[0]+1     
print(round(maxPercentage*100, 2),"% this is move № ", move)

[3.4286932e-03 3.0381036e-06 1.7301516e-06 1.9035144e-05 9.9933857e-01
 1.6817170e-06]
99.93 % this is move №  5


In [112]:
predictArr = seq_model.predict(predict_move_6)
print(predictArr[0])

maxPercentage = 0
for i in enumerate(predictArr[0]):
    if(i[1]>maxPercentage):
        maxPercentage=i[1]
        move=i[0]+1     
print(round(maxPercentage*100, 2),"% this is move № ", move)

[7.8054501e-07 9.9827719e-01 7.9352213e-07 4.9891582e-06 1.9779080e-04
 3.4066865e-03]
99.83 % this is move №  2
